TODO: 

* setup streams using python snowflake connector

In [1]:
import snowflake.connector
import os

/Users/jordanpacho/Desktop/Sigma_Labs/Coursework/Deloton-Exercise-Bikes/.venv/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [15]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
BATCH_SCHEMA= os.environ.get('BATCH_SCHEMA')
ROLE='ACCOUNTADMIN'

In [56]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=BATCH_SCHEMA,
    role=ROLE
)
cs = conn.cursor()

### Table Stream on RIDES table in Snowflake

* Table stream uses CDC on the RIDES table to track DDL statements
* Tracks all new _insert_ statements
* Data retention set to 1 day if data not consumed by DML statements

In [4]:
query = """
    CREATE STREAM rides_stream
    ON TABLE rides
"""
cs.execute(query)

In [85]:
query_stream = """
    SELECT *
    FROM rides_stream
"""

cs.execute(query_stream).fetch_pandas_all()

,RIDE_ID,USER_ID,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE,METADATA$ACTION,METADATA$ISUPDATE,METADATA$ROW_ID
0,179,4643,"{'epoch': 1665130779, 'fraction': 0}",520,21560.431,41.462367,39.638462,50.515385,94.238462,INSERT,False,707d50a95a34cb50ccc667ec6cf6394960a136d8


### Subset table for use in recent ride visualizations in the dashboard

* 

In [9]:
query = """
    CREATE OR REPLACE TABLE DELATON.ZOOKEEPERS_BATCH_PRODUCTION.RECENT_RIDES (
        USER_ID NUMBER(38,0) NOT NULL,
        RIDE_ID NUMBER(38,0) NOT NULL,
        GENDER VARCHAR(6) NOT NULL,
        AGE NUMBER(38,0) NOT NULL,
        BEGIN_TIMESTAMP TIMESTAMP_NTZ(9) NOT NULL,
        TOTAL_DURATION_SEC NUMBER(38,0) NOT NULL,
        TOTAL_POWER FLOAT NOT NULL,
        MEAN_POWER FLOAT NOT NULL,
        MEAN_RESISTANCE FLOAT NOT NULL,
        MEAN_RPM FLOAT NOT NULL,
        MEAN_HEART_RATE FLOAT NOT NULL
    )
"""

cs.execute(query)

### Query that join necessary data from USERS and joins it with RIDES data from the table stream

In [41]:
query = """
     WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth, CURRENT_DATE)/8766 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, CURRENT_DATE)
        )
        SELECT * FROM recent_rides_data


"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID,GENDER,AGE,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE
0,4617,74,Male,55.373032,"{'epoch': 1665077065, 'fraction': 0}",519,18022.659,34.72574,39.117534,44.055877,106.061657


In [38]:
query = """
    INSERT INTO recent_rides
        WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth,CURRENT_DATE)/8766 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, CURRENT_DATE)
        )
        SELECT * FROM recent_rides_data
"""

cs.execute(query)

In [37]:
query = """
   SELECT * 
    FROM rides_stream
"""

cs.execute(query).fetch_pandas_all()

,RIDE_ID,USER_ID,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE,METADATA$ACTION,METADATA$ISUPDATE,METADATA$ROW_ID
0,74,4617,"{'epoch': 1665077065, 'fraction': 0}",519,18022.659,34.72574,39.117534,44.055877,106.061657,INSERT,False,69f3f483a0b71c88fe44e9fb601e13ddfc9dc3b6


In [81]:
query = """
   SELECT * 
    FROM recent_rides
"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID,GENDER,AGE,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE
0,4640,170,Male,37,"{'epoch': 1665126089, 'fraction': 0}",520,23511.622,45.214658,40.580769,42.061538,108.325000
1,4640,168,Male,37,"{'epoch': 1665125048, 'fraction': 0}",520,23492.618,45.178112,40.761538,50.300000,119.540385
2,4640,169,Male,37,"{'epoch': 1665125571, 'fraction': 0}",516,28983.330,56.169244,40.906977,41.453488,27.420543
3,4640,167,Male,37,"{'epoch': 1665124525, 'fraction': 0}",520,23504.523,45.201006,40.823077,49.073077,100.859615
4,4640,166,Male,37,"{'epoch': 1665124004, 'fraction': 0}",519,23454.920,45.192524,40.639692,54.847784,91.171484
5,4640,165,Male,37,"{'epoch': 1665123481, 'fraction': 0}",520,25549.834,49.134296,40.792308,39.638462,117.963462
6,4640,163,Male,37,"{'epoch': 1665122512, 'fraction': 0}",444,20763.437,46.764498,42.468468,47.270270,104.932432
7,4639,161,Female,49,"{'epoch': 1665121466, 'fraction': 0}",520,17516.964,33.686469,38.342308,45.628846,116.630769
8,4639,162,Female,49,"{'epoch': 1665121989, 'fraction': 0}",520,19994.824,38.451585,38.419231,53.884615,76.413462
9,4639,160,Female,49,"{'epoch': 1665121158, 'fraction': 0}",306,12730.962,41.604451,44.418301,40.267974,129.290850


In [78]:
query = """
    INSERT INTO recent_rides
    VALUES (
        9999, 999, 'Male', 99, '2022-10-06 19:00:00', 99, 999, 9999999, 99, 99, 99
    )
"""

cs.execute(query)

In [68]:
query = """
    DELETE FROM recent_rides
            WHERE user_id = 9999
"""

cs.execute(query)

In [80]:
query = """
    DELETE FROM recent_rides
            WHERE TO_DATE(begin_timestamp) < DATEADD(HOUR, -12, CURRENT_DATE)
"""

cs.execute(query)

In [48]:
query = f"""
    CREATE TASK recent_ride_ingestion
        WAREHOUSE = {WAREHOUSE}
        SCHEDULE = '10 minute'
    AS
        INSERT INTO recent_rides
        WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth, CURRENT_DATE)/8766 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, CURRENT_DATE)
        )
        SELECT * FROM recent_rides_data
"""

cs.execute(query)

In [50]:
query = """
    DROP TASK recent_ride_updater
"""

cs.execute(query)

In [47]:
query = """
    DROP TASK recent_ride_ingestion
"""

cs.execute(query)

In [53]:
query = """
    DELETE FROM recent_rides
"""

cs.execute(query)

In [53]:
query = f"""
    CREATE TASK recent_ride_updater
        WAREHOUSE = {WAREHOUSE}
        AFTER recent_ride_ingestion
    AS
        DELETE FROM recent_rides
            WHERE TO_DATE(begin_timestamp) < DATEADD(HOUR, -12, CURRENT_DATE)
"""

cs.execute(query)

In [84]:
query = """
    ALTER TASK IF EXISTS recent_ride_ingestion RESUME
"""

cs.execute(query)

In [83]:
query = """
    ALTER TASK IF EXISTS recent_ride_updater RESUME
"""

cs.execute(query)

In [11]:
query = """
    EXECUTE TASK recent_ride_ingestion
"""

cs.execute(query)

In [25]:
query = f"""
    show grants to role {ROLE}
"""

cs.execute(query).fetchall()

[(datetime.datetime(2022, 9, 28, 4, 47, 14, 230000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'APPLY MASKING POLICY',
  'ACCOUNT',
  'RF77926',
  'ROLE',
  'ACCOUNTADMIN',
  'true',
  ''),
 (datetime.datetime(2022, 9, 28, 4, 47, 14, 226000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'CREATE ACCOUNT',
  'ACCOUNT',
  'RF77926',
  'ROLE',
  'ACCOUNTADMIN',
  'true',
  ''),
 (datetime.datetime(2022, 9, 28, 4, 47, 14, 238000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'CREATE CREDENTIAL',
  'ACCOUNT',
  'RF77926',
  'ROLE',
  'ACCOUNTADMIN',
  'true',
  ''),
 (datetime.datetime(2022, 9, 28, 4, 47, 14, 232000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'CREATE DATA EXCHANGE LISTING',
  'ACCOUNT',
  'RF77926',
  'ROLE',
  'ACCOUNTADMIN',
  'true',
  ''),
 (datetime.datetime(2022, 9, 28, 4, 47, 14, 234000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'CREATE FAILOVER

In [21]:
query = f"grant EXECUTE TASK on account to role {ROLE}"

cs.execute(query).fetchall()

[('Statement executed successfully.',)]